In [1]:
# import libraries
import pandas as pd
import numpy as np

## job class
each job requires input:
* job name
* the model type of the job
* what machine(s) is this job using (or possibly use)

each job contains fields:
* name: job name
* model: the model type of the job
* steps: what machine(s) is this job using (or possibly use)
* status: which step is this job currently on. -1 is not assigned yet, i=0,..,len(steps)-1 is working on step[0] or waiting for it to start
* current_machine: the machine this job is assigned to or working on, if any

In [2]:
class job:
    def __init__(self, job_name, model, use_machines):
        self.name = job_name
        self.model = model
        self.steps = use_machines
        self.status = -1 #not assigned to machines yet
        self.current_machine = None 

    def get_model(self):
        return self.model
    
    def get_name(self):
        return self.name
    
    def info(self):
        return (self.name, self.status)
    
    def get_position(self):
        return self.status

    def get_next_machine(self):
        if self.status >= len(self.steps)-1:
            return None
        else:
            machines = self.steps[self.status+1]
        
        if len(machines) == 1: return machines[0]
        else:
            for m in machines:
                if m.free(): return m
        sorted_machines = machines.sort(key=lambda m: len(m.waitlist))
        return machines[0]
    
    def move_to_next_machine(self):
        m = self.get_next_machine()
        if m:
            self.status += 1
            self.current_machine = m

## machine class
each machine requires input:
* machine name
* runtimes for each model (dictionary)
* setup times for each model (dictionary)

each machine has fields:
* machine name
* runtimes for each model
* setup times for each model
* the model the machine was/is currently working on, if any
* the job that is assigned to this machine or the job currently running, if any
* time to complete current job (or the job assigned to start in the future)
* the jobs waiting on this machine


In [3]:
class machine:
    def __init__(self, machine_name, runtimes, setup):
        self.name = machine_name
        self.runtimes = runtimes
        self.setup = setup
        self.status = 0 # 0 is not working, 0.5 is setup for new model, 1 is working
        self.previous_model = None
        self.current_job = None
        self.time_to_completion = 0
        self.waitlist = []
    
    def shutdown(self):
        self.status = 0
    
    def free(self):
        if not self.current_job or self.time_to_completion == 0: return True
        return False
    
    def get_current_job(self):
        return self.current_job
    
    def get_current_job_name(self):
        j = self.get_current_job()
        if j: return j.get_name()
        else: return ""
    
    def get_status(self):
        return self.status
    
    def get_time_to_completion(self):
        return self.time_to_completion
    
    def show_status(self):
        if self.status == 0.5: 
            return ("setup")
        elif self.status == 1:
            return ("working")
        else:
            return ("not working")
    
    def get_info(self):
        if self.status == 0:
            if self.current_job:
                print("machine {} is not working now, waiting to start job {}".format(self.name, self.current_job.get_name()))
                return (self.name, self.status, self.current_job.name, self.time_to_completion)
            else:
                print("machine {} is not working now".format(self.name))
                return (self.name, self.status,"", self.time_to_completion)
        else:
            print("machine {} is {} for model {}".format(self.name, self.show_status(), self.current_job.get_name()))
            return (self.name, self.status, self.current_job.name, self.time_to_completion)
    
    def get_waitlist(self):
        return self.waitlist
    
    
    def get_job_setup_time(self, model):
        return self.setup[model]
    
    
    def get_job_runtime(self, model):
        return self.runtimes[model]
    
    def if_in_use(self):
        if not self.current_job and self.time_to_completion <=0: 
            return False
        elif self.current_job and self.time_to_completion <=0:
            print("ERROR!!! machine {} is has job {} but has 0 or negative remaining runtime".format(self.name, self.current_job.name))
        return True
    
    def add_job(self, job, remaining_time_of_day):
        if self.current_job:
            self.waitlist.append(job)
        else:
            j = self.try_start_next_job(job, remaining_time_of_day)
            # if could not started the job, append it to waitlist
            if j and j not in self.waitlist: 
                self.waitlist.append(j)
            
    # ignore previous current_job and time to complete info and reset
    def start_job(self, job):
        
        if job in self.waitlist:
            self.waitlist.remove(job)
        self.current_job = job
        new_model = job.get_model()
        self.time_to_completion = self.get_job_runtime(new_model)
        self.previous_model = new_model
        self.status = 1
        #print("machine {} just started new job {}".format(self.name, job.get_name()))
    
    def complete_job(self):
        j = self.current_job
        self.current_job = None
        self.time_to_completion = 0
        self.status = 0
        return j
    
    def job_need_time(self, job):
        _, _, setup_time = self.need_setup(job)
        return self.get_job_runtime(job.get_model())+setup_time
    
    # compare possible setup time and runtime, pick the shortest & first arrived one
    def get_next_job(self):
        if not self.waitlist: 
            return None
        elif len(self.waitlist) == 1:
            j = self.waitlist[0]
            return j
        else:
            j_times = [self.job_need_time(j) for j in self.waitlist]
            next_job_idx = np.argsort(j_times)[0]
            j = self.waitlist[next_job_idx]
            #self.waitlist = self.waitlist[:next_job_idx]+self.waitlist[next_job_idx+1:]
            return j
    
    # input: job; output: (if_need_setup, new_model, setup_time)
    def need_setup(self, job):
        new_model = job.get_model()
        if self.previous_model and new_model != self.previous_model:
            return (True, new_model, self.get_job_setup_time(new_model))
        return (False, None, 0)
    
    def setup_new_model(self, job, new_model, setup_time):
        self.status = 0.5
        self.time_to_completion = setup_time
        self.previous_model = new_model
        self.current_job = job
    
    def try_start_next_job(self, j, remaining_time_of_day):
        if_need_setup, new_model, setup_time = self.need_setup(j)
        j_runtime = self.get_job_runtime(j.get_model())
        # need setup and have time to do it
        if if_need_setup and setup_time <= remaining_time_of_day:
            self.setup_new_model(j, new_model, setup_time)
            # have not started the job yet, return the job
            return j
        
        # does neet setup and have time to run it
        elif not if_need_setup and j_runtime <= remaining_time_of_day:
            self.start_job(j)

        # need setup and does not have time to do it, done for the day
        # or does not need setup but need more runtime than given
        else:
            self.status = 0
            self.current_job = None
            self.time_to_completion = 0
            # can't start the job, return the job
            return j
        
        return None
    
    # update machine status after time t
    def update(self, t, remaining_time_of_day):
        completed_job = None
        #if self.current_job:
        # the machine was not working, now could start work
        if self.status == 0:
            # start possible work assigned from yesterday
            if self.current_job and self.time_to_completion <= remaining_time_of_day:
                self.start_job(self.current_job)
            # or if nothing was assigned, try start a new job
            elif not self.current_job:
                j = self.get_next_job()
                if j:
                    self.try_start_next_job(j, remaining_time_of_day)

            # if have an assigned job but has nothing to do
            else:
                pass
        # check if completed current job
        elif self.time_to_completion - t <= 0 and self.status == 1:
            completed_job = self.complete_job()
            j = self.get_next_job()
            if j:
                self.try_start_next_job(j, remaining_time_of_day)


        # if just finished setup, check if having time for next operation
        elif self.time_to_completion - t <= 0 and self.status == 0.5:
            # if have time to run model:
            j_runtime = self.get_job_runtime(self.current_job.get_model())
            if j_runtime <= remaining_time_of_day:
                self.time_to_completion = 0
                self.start_job(self.current_job)
            else:
                #not have time to run model
                self.status = 0
                self.time_to_completion = self.get_job_runtime(self.current_job.get_model())

        else:
            self.time_to_completion -= t


        return completed_job
            

## factory class
requires input:
* machines
* how many hours is this factory working per day
* (optional) initial jobs   <-- some bugs here, please use release_jobs function instead

In [4]:
class factory:
    def __init__(self, machines, working_hours, jobs = []):
        self.machines = machines
        self.jobs = jobs
        self.working_hours = working_hours
        
    def get_status(self):
        df = pd.DataFrame(columns=["machines","status","assigned_job","time_to_completion","num_jobs_waiting"])
        df['machines'] = [m.name for m in self.machines]
        df['status'] = [m.show_status() for m in self.machines]
        df['assigned_job'] = [m.get_current_job_name() for m in self.machines]
        df['time_to_completion'] = [m.get_time_to_completion() for m in self.machines]
        df['num_jobs_waiting'] = [len(m.get_waitlist()) for m in self.machines]
         
        return df
    
    def print_waiting_jobs(self):
        for m in self.machines:
            w = [j.get_name() for j in m.waitlist]
            if m.get_current_job_name(): w.append(m.get_current_job_name())
            print("machine {} jobs waitlist: {}".format(m.name,w))
    
    def release_jobs(self, jobs, remaining_time_of_day):
        finished_products = []
        for j in jobs:
            if j not in self.jobs: self.jobs.append(j)
            
            m = j.get_next_machine()
            
            if m: 
                j.move_to_next_machine()
                m.add_job(j, remaining_time_of_day)
            else:
                # completed all steps
                if j in self.jobs: self.jobs.remove(j)
                finished_products.append(j.get_name())
                #print("job {} has finished all steps.".format(j.get_name()))
        
        # return list of finished jobs
        return finished_products
    
    def workday(self, remaining_time_of_day = None, t = 0.5):
        if not remaining_time_of_day: remaining_time_of_day = self.working_hours
        setup_times = dict.fromkeys(self.machines, 0)
        runtimes = dict.fromkeys(self.machines, 0)
        finished_products= []
        while remaining_time_of_day > 0:
            completed_jobs = []
            for m in self.machines:
                # keep track of runtime and setup times
                if m.get_status() == 1: 
                    runtimes[m] += t
                elif m.get_status() == 0.5: 
                    setup_times[m] += t
                
                j = m.update(t, remaining_time_of_day)
                if j: 
                    completed_jobs.append(j)
                    print("machine {} completed job {} at time {}".format(m.name, j.get_name(), self.working_hours-remaining_time_of_day))
                
            finished_products += self.release_jobs(completed_jobs, remaining_time_of_day)
            
            remaining_time_of_day -= t
        
        # shutdown all machines at the end of day
        for m in self.machines:
            m.shutdown()
        
        #print today's finished products (jobs that completed all steps)
        print("finished products : {}".format(finished_products))
        
        #print job waitlist for each machine
        self.print_waiting_jobs()
        
        #print("machines {}".format([m.name for m in self.machines]))
        print("total setup time for each machine: {}".format(setup_times.values()))
        print("total runtime for each machine: {}".format(runtimes.values()))
            
        #return self.get_status()
    
    
    
    

## a helper function to create jobs

In [5]:
# create jobs with name format "model_x" where x = 1,2, ...
def create_job(name, model=None):
    if not model: model = name[:-2]
    return job(name, model, machines_dict[model])

# example: create 10 B15 jobs
# B15_jobs = [create_job("B15_"+str(i)) for i in range(10)]

# How to use this model:

## 1. define the settings for machines and models

In [6]:
# settings
# chucker 1, 2, 3
runtimes1 = {"D20":4.5,"D25":4.5,"B15":6,"E26":6,"C17":3}
setup1 = {"D20":1.5,"D25":1.5,"B15":1.5,"E26":1.5,"C17":1.5}
c1,c2,c3 = machine("Chucker1", runtimes1, setup1),machine("Chucker2", runtimes1, setup1),machine("Chucker3", runtimes1, setup1)

# need change following values
runtimes2 = {"D20":8,"D25":8,"B15":8,"E26":9,"C17":8}
setup2 = {"D20":1.5,"D25":1.5,"B15":1.5,"E26":1.5,"C17":1.5}
ca,cb,cc,cd= machine("ChuckerA", runtimes2, setup2),machine("ChuckerB", runtimes2, setup2),machine("ChuckerC", runtimes2, setup2),machine("ChuckerD", runtimes2, setup2)


runtimes3 = {"D20":3,"D25":3,"B15":4,"E26":4.5,"C17":4}
setup3 = {"D20":2,"D25":2,"B15":2,"E26":2,"C17":2}
m1, m2 = machine("Mill1", runtimes3, setup3),machine("Mill2", runtimes3, setup3)

runtimes4 = {"D20":2,"D25":2,"B15":1.5,"E26":2,"C17":1.5,"F35":3}
setup4 = {"D20":3,"D25":3,"B15":3,"E26":3,"C17":3, "F35":3}
nd = machine("Naco Drill", runtimes4, setup4)

# create list for all machies
machines = [c1,c2,c3,ca,cb,cc,cd,m1,m2,nd]

# models of the jobs
models = ["D20","D25","B15","E26","C17","F35"]
machines_dict = {"D25":[[c2], [cb,cc], [m1], [nd]], 
                "D20":[[c3], [cd], [m2], [nd]],
                "B15":[[c3], [cc,cd],[m2],[nd]],
                "E26":[[c3], [cb],[m1],[nd]],
                "C17":[[c3], [ca,cb],[m1],[nd]],
                "F35":[[nd]]}



## 2. create factory, create jobs and release jobs
## 3. run factory.workday() to make it work for a day

In [7]:
# example for run 10 B15 jobs and 10 D25 jobs for 10 days (2 weeks)

B15_jobs = [create_job("B15_"+str(i)) for i in range(10)]
D25_jobs = [create_job("D25_"+str(i)) for i in range(10)]

print("day 1")
my_factory = factory(machines, 16)
my_factory.release_jobs(B15_jobs,16)
my_factory.release_jobs(D25_jobs,16)
my_factory.workday()


print("day 2")
my_factory.workday()
print("day 3")
my_factory.workday()
print("day 4")
my_factory.workday()
print("day 5")
my_factory.workday()

# next week
print("day 6")
my_factory.workday()
print("day 7")
my_factory.workday()
print("day 8")
my_factory.workday()
print("day 9")
my_factory.workday()
print("day 10")
my_factory.workday()

day 1
machine Chucker2 completed job D25_0 at time 4.0
machine Chucker3 completed job B15_0 at time 5.5
machine Chucker2 completed job D25_1 at time 8.5
machine Chucker3 completed job B15_1 at time 11.5
machine ChuckerB completed job D25_0 at time 12.0
machine Chucker2 completed job D25_2 at time 13.0
machine ChuckerC completed job B15_0 at time 13.5
machine Mill1 completed job D25_0 at time 15.0
finished products : []
machine Chucker1 jobs waitlist: []
machine Chucker2 jobs waitlist: ['D25_3', 'D25_4', 'D25_5', 'D25_6', 'D25_7', 'D25_8', 'D25_9']
machine Chucker3 jobs waitlist: ['B15_2', 'B15_3', 'B15_4', 'B15_5', 'B15_6', 'B15_7', 'B15_8', 'B15_9']
machine ChuckerA jobs waitlist: []
machine ChuckerB jobs waitlist: ['D25_1', 'D25_2']
machine ChuckerC jobs waitlist: []
machine ChuckerD jobs waitlist: ['B15_1']
machine Mill1 jobs waitlist: []
machine Mill2 jobs waitlist: ['B15_0']
machine Naco Drill jobs waitlist: ['D25_0']
total setup time for each machine: dict_values([0, 0, 0, 0, 0, 

### The output will print when a machine completed a job, and all the finished products and jobs waiting at each machines at the end of day